## Pruebas para generar las columnas de las matrices

In [5]:
import pyodbc
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

def get_sql_connection():
    """
    Establece la conexión con SQL Server.
    Configura los datos de conexión antes de ejecutar.
    """
    try:
        conn = pyodbc.connect(
            "DRIVER={ODBC Driver 17 for SQL Server};"
            f"SERVER={os.getenv('DB_SERVER')};"
            f"DATABASE={os.getenv('DATABASE')};"
            f"UID={os.getenv('DB_USER')};"
            f"PWD={os.getenv('DB_PASSWORD')}"
        )
        print("Conexión exitosa a la base de datos.")
        return conn
    except pyodbc.Error as e:
        print(f"Error al conectar a la base de datos: {str(e)}")
        raise

def execute_query(query):
    """
    Ejecuta el query en la base de datos y retorna un DataFrame.
    """
    conn = get_sql_connection()
    try:
        df = pd.read_sql(query, conn)
    except Exception as e:
        print(f"Error al ejecutar el query: {str(e)}")
        df = pd.DataFrame()  # Retorna un DataFrame vacío en caso de error
    finally:
        conn.close()
    return df

def get_inventario_matriz():
    """
    Extrae todos los registros de la tabla InventarioMatriz.
    """
    query = """
    SELECT 
        id_politica_base_riesgo,
        subsegmento,
        negocio,
        estado,
        cobertura
    FROM InventarioMatriz
    """
    return execute_query(query)

def get_matrices_base_riesgo():
    """
    Extrae todos los registros de la tabla MatricesBaseRiesgo.
    """
    query = """
    SELECT
        id_politica_base_riesgo,
        concatenado,
        segmento,
        permanencia,
        factor_prov,
        clasificacion,
        tipo_matriz
    FROM MatrizBaseRiesgo
    """
    return execute_query(query)

def unir_dataframes_merge():
    """
    Extrae los datos de ambas tablas, los unifica usando pd.merge() y convierte todos los campos a mayúsculas.
    """
    df_inventario = get_inventario_matriz()
    df_matrices = get_matrices_base_riesgo()
    
    # Se une utilizando la columna en común 'id_politica_base_riesgo'
    df_merge = pd.merge(df_inventario, df_matrices, 
                        on="id_politica_base_riesgo", 
                        how="inner")  # Cambia 'inner' por 'left' o 'outer' según lo requieras
    
    # Convertir todos los campos a mayúsculas
    df_merge = df_merge.applymap(lambda x: x.upper() if isinstance(x, str) else x)
    df_merge['factor_prov'] = df_merge['factor_prov'].apply(lambda x: x/100)
    
    
    return df_merge



In [6]:
df_merge = unir_dataframes_merge()
df_merge

Conexión exitosa a la base de datos.
Conexión exitosa a la base de datos.


C:\Users\prac.planeacionfi\AppData\Local\Temp\ipykernel_22992\2033789348.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\prac.planeacionfi\AppData\Local\Temp\ipykernel_22992\2033789348.py:86: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_merge = df_merge.applymap(lambda x: x.upper() if isinstance(x, str) else x)


,id_politica_base_riesgo,subsegmento,negocio,estado,cobertura,concatenado,segmento,permanencia,factor_prov,clasificacion,tipo_matriz
0,1,None,FPT,OBSOLETO,None,FPTOBSOLETO1.MENOR DE 90 DIAS,None,1.MENOR DE 90 DIAS,0.00,BAJO,MATRIZ NATURACO
1,2,None,FPT,OBSOLETO,None,FPTOBSOLETO2.ENTRE 90 Y 180 DIAS,None,2.ENTRE 90 Y 180 DIAS,0.15,MEDIO,MATRIZ NATURACO
2,3,None,FPT,OBSOLETO,None,FPTOBSOLETO3.ENTRE 180 Y 270 DIAS,None,3.ENTRE 180 Y 270 DIAS,0.30,MEDIO,MATRIZ NATURACO
3,4,None,FPT,OBSOLETO,None,FPTOBSOLETO4.ENTRE 270 Y 360 DIAS,None,4.ENTRE 270 Y 360 DIAS,0.50,MEDIO-ALTO,MATRIZ NATURACO
4,5,None,FPT,OBSOLETO,None,FPTOBSOLETO5.ENTRE 360 Y 540 DIAS,None,5.ENTRE 360 Y 540 DIAS,1.00,MUY ALTO,MATRIZ NATURACO
...,...,...,...,...,...,...,...,...,...,...,...
1150,1151,TOLL,None,None,5.MAYOR O IGUAL A 360 DIAS,EXPERTOS LOCALESTOLL5.MAYOR O IGUAL A 360 DIAS...,EXPERTOS LOCALES,3.ENTRE 180 Y 270 DIAS,0.80,MUY ALTO,PVA 4 A 6 MESES
1151,1152,TOLL,None,None,5.MAYOR O IGUAL A 360 DIAS,EXPERTOS LOCALESTOLL5.MAYOR O IGUAL A 360 DIAS...,EXPERTOS LOCALES,4.ENTRE 270 Y 360 DIAS,0.80,MUY ALTO,PVA 4 A 6 MESES
1152,1153,TOLL,None,None,5.MAYOR O IGUAL A 360 DIAS,EXPERTOS LOCALESTOLL5.MAYOR O IGUAL A 360 DIAS...,EXPERTOS LOCALES,5.ENTRE 360 Y 540 DIAS,0.80,MUY ALTO,PVA 4 A 6 MESES
1153,1154,TOLL,None,None,5.MAYOR O IGUAL A 360 DIAS,EXPERTOS LOCALESTOLL5.MAYOR O IGUAL A 360 DIAS...,EXPERTOS LOCALES,6.ENTRE 540 Y 720 DIAS,0.80,MUY ALTO,PVA 4 A 6 MESES


In [ ]:
def insert_marks() -> Dict[str, str]:
    """
    Retorna un diccionario con el mapeo de marcas QM a marcas concatenadas.
    """
    # TODO: Implementar el diccionario de marcas según la lógica de negocio
    return {
        "ACCESORIOS": "ACCESORIOS",
        "ADIDAS": "ADIDAS",
        "AGATHA RUIZ DE LA PRADA": "AGATHA RUIZ DE LA PRADA",
        "ALICORP": "ALICORP",
        "AMAZON": "AMAZON",
        "AMWAY": "AMWAY",
        "ARDEN FOR MEN": "AFM/CFM",
        "AVON": "AVON",
        "BALANCE": "BALANCE",
        "BANCO PREBEL": "BANCO PREBEL",
        "BEAUTYHOLICS": "UTOPICK",
        "BIO OIL": "BIO OIL",
        "BIOTECNIK": "BIOTECNIK",
        "BURTS_BEES": "BURT'S BEES",
        "CADIVEU": "CADIVEU",
        "calculateA": "calculateA",
        "CATRICE": "CATRICE",
        "CONNECT FOR MEN": "AFM/CFM",
        "COSMETRIX": "COSMETRIX",
        "COVER GIRL": "COVER GIRL",
        "DIAL": "DIAL",
        "DOVE": "DOVE",
        "DYCLASS": "DYCLASS",
        "ECAR": "ECAR",
        "EL EXITO": "EL EXITO",
        "ELIZABETH ARDEN": "ELIZABETH ARDEN",
        "ESSENCE": "ESSENCE",
        "FAMILIA": "FAMILIA",
        "FEBREZE": "FEBREZE",
        "FISA": "FISA",
        "HASK": "HASK",
        "HENKEL": "HENKEL",
        "HERBAL ESSENCES": "HERBAL ESSENCES",
        "IMPORTADOS PROCTER": "IMPORTADOS PROCTER",
        "JERONIMO MARTINS": "JERONIMO MARTINS",
        "KANABECARE": "KANABECARE",
        "KIMBERLY": "KIMBERLY",
        "KOBA": "D1",
        "L&G ASOCIADOS": "L&G ASOCIADOS",
        "LA POPULAR": "LA POPULAR",
        "LEONISA": "LEONISA",
        "LOCATEL": "LOCATEL",
        "LOREAL": "LOREAL",
        "LOVE, BEAUTY AND PLANET": "LOVE, BEAUTY AND PLANET",
        "MAUI": "MAUI",
        "MAX FACTOR": "MAX FACTOR",
        "MAX FACTOR EXPORTACIÓN": "MAX FACTOR",
        "MAX FACTOR GLOBAL": "MAX FACTOR",
        "MF COL + EXP": "MAX FACTOR",
        "MF GLOBAL": "MAX FACTOR",
        "MILAGROS": "MILAGROS",
        "MONCLER": "MONCLER",
        "MORROCCANOIL": "MORROCCANOIL",
        "NATURA": "NATURA",
        "NATURAL PARADISE": "NATURAL PARADISE",
        "NIVEA": "NIVEA",
        "NOPIKEX": "NOPIKEX",
        "NOVAVENTA FPT": "NOVAVENTA FPT",
        "NUDE": "NUDE",
        "OGX": "OGX",
        "OLAY": "OLAY",
        "OMNILIFE": "OMNILIFE",
        "OTRAS": "OTRAS",
        "PREBEL": "PREBEL",
        "QVS": "ACCESORIOS",
        "SALLY HANSEN": "SALLY HANSEN",
        "SIN ASIGNAR": "SIN ASIGNAR",
        "SOLLA": "SOLLA",
        "ST. IVES": "ST. IVES",
        "UBU": "ACCESORIOS",
        "UNILEVER": "UNILEVER",
        "VENTA DIRECTA COSMÉTICOS": "VENTA DIRECTA COSMÉTICOS",
        "VITÚ": "VITÚ",
        "VITÚ  EXPORTACIÓN": "VITÚ",
        "WELLA CONSUMO": "WELLA CONSUMO",
        "WELLA PROFESSIONAL": "WELLA PROFESSIONAL",
        "YARDLEY": "YARDLEY",
        "CATÁLOGO DE PRODUCTOS": "CATÁLOGO DE PRODUCTOS",
        "D1": "D1",
        "WORMSER": "WORMSER",
        "PROCTER AND GAMBLE": "P&G",
        "DAVINES": "DAVINES",
        "LA FABRIL": "LA FABRIL",
        "REVOX": "REVOX",
        "TENDENCIAS AB": "TENDENCIAS AB",
    }


def insert_subsegmentacion() -> Dict[str, str]:


    """
    RETORNA UN DICCIONARIO CON EL MAPEO DE MARCAS QM A SUBSEGMENTACIÓN.
    """
    return {
        "ACCESORIOS": "OTROS",
        "ADIDAS": "OTROS",
        "AGATHA RUIZ DE LA PRADA": "OTROS",
        "ALICORP": "FULL",
        "AMAZON": "RETAILERS",
        "AMWAY": "SISTEMA DE VENTAS",
        "ARDEN FOR MEN": "OTROS",
        "AVON": "FULL",
        "BALANCE": "FULL",
        "BANCO PREBEL": "FULL",
        "BEAUTYHOLICS": "OTROS",
        "BIO OIL": "OTROS",
        "BIOTECNIK": "FULL",
        "BURTS_BEES": "OTROS",
        "CADIVEU": "PROFESIONALES",
        "CALA": "FULL",
        "CATRICE": "OTROS",
        "CONNECT FOR MEN": "OTROS",
        "COSMETRIX": "OTROS",
        "COVER GIRL": "OTROS",
        "DIAL": "RETAILERS",
        "DOVE": "OTROS",
        "DYCLASS": "SISTEMA DE VENTAS",
        "ECAR": "FULL",
        "EL EXITO": "RETAILERS",
        "ELIZABETH ARDEN": "OTROS",
        "ESSENCE": "OTROS",
        "FAMILIA": "FULL",
        "FEBREZE": "OTROS",
        "FISA": "FULL",
        "HASK": "OTROS",
        "HENKEL": "FULL",
        "HERBAL ESSENCES": "OTROS",
        "IMPORTADOS PROCTER": "OTROS",
        "JERONIMO MARTINS": "RETAILERS",
        "KANABECARE": "OTROS",
        "KIMBERLY": "FULL",
        "KOBA": "RETAILERS",
        "L&G ASOCIADOS": "FULL",
        "LA POPULAR": "RETAILERS",
        "LEONISA": "SISTEMA DE VENTAS",
        "LOCATEL": "RETAILERS",
        "LOREAL": "FULL",
        "LOVE, BEAUTY AND PLANET": "OTROS",
        "MAUI": "OTROS",
        "MAX FACTOR": "OTROS",
        "MAX FACTOR EXPORTACIÓN": "OTROS",
        "MAX FACTOR GLOBAL": "OTROS",
        "MILAGROS": "SISTEMA DE VENTAS",
        "MONCLER": "RETAILERS",
        "MORROCCANOIL": "PROFESIONALES",
        "NATURA": "FULL",
        "NATURAL PARADISE": "OTROS",
        "NIVEA": "FULL",
        "NOPIKEX": "OTROS",
        "NOVAVENTA FPT": "SISTEMA DE VENTAS",
        "NUDE": "NUDE",
        "OGX": "OTROS",
        "OLAY": "OTROS",
        "OMNILIFE": "SISTEMA DE VENTAS",
        "OTRAS": "OTROS",
        "PREBEL": "OTROS",
        "QVS": "OTROS",
        "SALLY HANSEN": "OTROS",
        "SIN ASIGNAR": "FULL",
        "SOLLA": "FULL",
        "ST. IVES": "OTROS",
        "UBU": "OTROS",
        "UNILEVER": "TOLL",
        "VENTA DIRECTA COSMÉTICOS": "FULL",
        "VITÚ": "OTROS",
        "VITÚ  EXPORTACIÓN": "OTROS",
        "WELLA CONSUMO": "OTROS",
        "WELLA PROFESSIONAL": "PROFESIONALES",
        "YARDLEY": "OTROS",
        "D1": "RETAILERS",
        "CATÁLOGO DE PRODUCTOS": "RETAILERS",
        "WORMSER": "RETAILERS",
        "PROCTER AND GAMBLE": "FULL",
        "DAVINES": "OTROS",
        "LA FABRIL": "OTROS",
        "REVOX": "OTROS",
        "TENDENCIAS AB": "RETAILERS"
    }


def insert_segments() -> Dict[str, str]:
    """
    Retorna un diccionario con el mapeo de marcas QM a segmentaciones.
    """
    # TODO: Implementar el diccionario de segmentaciones según la lógica de negocio
    return {
        "OTROS CLIENTES DO": "DUEÑOS DE CANAL",
        "MARKETING PERSONAL": "DUEÑOS DE CANAL",
        "OMNILIFE": "DUEÑOS DE CANAL",
        "JERONIMO MARTINS": "DUEÑOS DE CANAL",
        "LEONISA": "DUEÑOS DE CANAL",
        "LOCATEL": "DUEÑOS DE CANAL",
        "NOVAVENTA": "DUEÑOS DE CANAL",
        "EL ÉXITO": "DUEÑOS DE CANAL",
        "MILAGROS ENTERPRISE": "DUEÑOS DE CANAL",
        "LA POPULAR": "DUEÑOS DE CANAL",
        "D1": "DUEÑOS DE CANAL",
        "USA": "DUEÑOS DE CANAL",
        "USA": "DUEÑOS DE CANAL",
        "EL EXITO": "DUEÑOS DE CANAL",
        "NOVAVENTA FPT": "DUEÑOS DE CANAL",
        "MILAGROS": "DUEÑOS DE CANAL",
        "WORMSER": "DUEÑOS DE CANAL",
        "TENDENCIAS AB": "DUEÑOS DE CANAL",
        "LA FABRIL": "DUEÑOS DE CANAL",
        "UNILEVER": "EXPERTOS LOCALES",
        "NATURA": "EXPERTOS LOCALES",
        "BIOTECNIK": "EXPERTOS LOCALES",
        "NIVEA": "EXPERTOS LOCALES",
        "BRITO": "EXPERTOS LOCALES",
        "AVON": "EXPERTOS LOCALES",
        "OTROS EXPERTOS LOCALES": "EXPERTOS LOCALES",
        "ALICORP": "EXPERTOS LOCALES",
        "SOLLA": "EXPERTOS LOCALES",
        "ECAR": "EXPERTOS LOCALES",
        "FISA": "EXPERTOS LOCALES",
        "KIMBERLY": "EXPERTOS LOCALES",
        "BELCORP": "EXPERTOS LOCALES",
        "AMWAY": "EXPERTOS LOCALES",
        "PROCTER AND GAMBLE": "EXPERTOS LOCALES",
        "HENKEL": "EXPERTOS LOCALES",
        "DIAL": "EXPERTOS LOCALES",
        "BEIERSDORF": "EXPERTOS LOCALES",
        "OTROS EXPERTOS LOCALES": "EXPERTOS LOCALES",
        "FAMILIA": "EXPERTOS LOCALES",
        "BALANCE": "EXPERTOS LOCALES",
        "MAX FACTOR": "EXPERTOS NO LOCALES",
        "DYCLASS": "EXPERTOS NO LOCALES",
        "WELLA CONSUMO": "EXPERTOS NO LOCALES",
        "BIO OIL": "EXPERTOS NO LOCALES",
        "OGX": "EXPERTOS NO LOCALES",
        "COVER GIRL": "EXPERTOS NO LOCALES",
        "WELLA PROFESSIONAL": "EXPERTOS NO LOCALES",
        "ADIDAS": "EXPERTOS NO LOCALES",
        "ACCESORIOS": "EXPERTOS NO LOCALES",
        "BURTS_BEES": "EXPERTOS NO LOCALES",
        "NOPIKEX": "EXPERTOS NO LOCALES",
        "QVS": "EXPERTOS NO LOCALES",
        "UBU": "EXPERTOS NO LOCALES",
        "ESSENCE": "EXPERTOS NO LOCALES",
        "MORROCCANOIL": "EXPERTOS NO LOCALES",
        "HASK": "EXPERTOS NO LOCALES",
        "HERBAL ESSENCES": "EXPERTOS NO LOCALES",
        "LOVE, BEAUTY AND PLANET": "EXPERTOS NO LOCALES",
        "CATRICE": "EXPERTOS NO LOCALES",
        "NATURAL PARADISE": "EXPERTOS NO LOCALES",
        "OLAY": "EXPERTOS NO LOCALES",
        "MID": "EXPERTOS NO LOCALES",
        "SECRET": "EXPERTOS NO LOCALES",
        "FEBREZE": "EXPERTOS NO LOCALES",
        "TAMPAX": "EXPERTOS NO LOCALES",
        "OFCORSS C.I HERMECO": "EXPERTOS NO LOCALES",
        "CADIVEU": "EXPERTOS NO LOCALES",
        "MAX FACTOR GLOBAL": "EXPERTOS NO LOCALES",
        "SEBASTIAN": "EXPERTOS NO LOCALES",
        "AFFRESH": "EXPERTOS NO LOCALES",
        "COSMETRIX": "EXPERTOS NO LOCALES",
        "INCENTIVOS MAX FACTOR": "EXPERTOS NO LOCALES",
        "OTROS EXPERTOS NO LOCALES": "EXPERTOS NO LOCALES",
        "DAVINES": "EXPERTOS NO LOCALES",
        "P&G": "EXPERTOS NO LOCALES",
        "REVOX": "EXPERTOS NO LOCALES",
        "UTOPICK": "EXPERTOS NO LOCALES",
        "IMPORTADOS PROCTER": "EXPERTOS NO LOCALES",
        "ST. IVES": "EXPERTOS NO LOCALES",
        "ARDEN FOR MEN": "MARCAS PROPIAS",
        "NUDE": "MARCAS PROPIAS",
        "ELIZABETH ARDEN": "MARCAS PROPIAS",
        "YARDLEY": "MARCAS PROPIAS",
        "VITÚ": "MARCAS PROPIAS",
        "PREBEL": "MARCAS PROPIAS",
        "OTRAS MP": "MARCAS PROPIAS",
        "AFM/CFM": "MARCAS PROPIAS",
        "BODY CLEAR": "NO APLICA",
        "OTRAS": "NO APLICA",
        "GILLETTE": "NO APLICA",
        "L&G ASOCIADOS": "NO APLICA",
        "CATÁLOGO DE PRODUCTOS": "NO APLICA",
        "HINODE": "NO APLICA",
        "PFIZER": "NO APLICA",
        "CONTEXPORT DISNEY": "NO APLICA",
        "SYSTEM PROFESSIONAL": "NO APLICA",
        "WELONDA": "NO APLICA",
        "SIN ASIGNAR": "NO APLICA",
        "CATÁLOGO DE PRODUCTOS": "NO APLICA",
    }


def calculate_rango_permanencia(row: pd.Series) -> str:
    """
    Calcula el rango de permanencia basado en las condiciones especificadas.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Rango de permanencia calculado
    """
    lote = row.get("LOTE", None)
    permanencia = row.get("PERMANENCIA", None)
    rango_permanencia = row.get("RANGO DE PERMANENCIA", None)

    if lote == "222222":
        return "1.MENOR DE 90 DIAS"
    elif permanencia == 0 and rango_permanencia == "5.MAYOR O IGUAL A 360 DIAS":
        return "5.ENTRE 360 Y 540 DIAS"
    elif rango_permanencia == "5.MAYOR O IGUAL A 360 DIAS":
        if permanencia < 540:
            return "5.ENTRE 360 Y 540 DIAS"
        elif permanencia < 720:
            return "6.ENTRE 540 Y 720 DIAS"
        else:
            return "7.MAYOR DE 720 DIAS"
    else:
        return rango_permanencia


def calculate_status_cons(row: pd.Series) -> str:
    """
    Calcula el estado de consumo basado en las condiciones de vencimiento, bloqueo y obsolescencia.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Estado calculado (VENCIDO, BLOQUEADO, OBSOLETO, PAV o DISPONIBLE)
    """
    rango_prox_vencer = row.get("RANGO PRÓX.VENCER MM")
    valor_BLOQUEADO = row.get("VALOR BLOQUEADO MM")
    valor_OBSOLETO = row.get("VALOR OBSOLETO")

    if rango_prox_vencer == "VENCIDO":
        return "VENCIDO"
    elif valor_BLOQUEADO != 0:
        return "BLOQUEADO"
    elif valor_OBSOLETO != 0:
        return "OBSOLETO"
    elif rango_prox_vencer in ["1.PAV 3 MESES", "2.PAV 4 A 6 MESES"]:
        return "PAV"
    else:
        return "DISPONIBLE"


def calculate_valor_def(row: pd.Series) -> float:
    """
    Calcula el valor definitivo basado en el estado de consumo.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        float: Valor definitivo calculado
    """
    status_cons = row.get("STATUS CONS")
    valor_BLOQUEADO = row.get("VALOR BLOQUEADO MM")
    valor_total = row.get("VALOR TOTAL MM")

    return valor_BLOQUEADO if status_cons == "BLOQUEADO" else valor_total


def calculate_rango_obsolescencia(row: pd.Series) -> str:
    """
    Calcula el rango de obsolescencia basado en las fechas de entrada y obsolescencia.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Rango de obsolescencia calculado
    """
    status_cons = row.get("STATUS CONS")
    fecha_entrada = row.get("FECHA ENTRADA")
    fecha_OBSOLETO = row.get("FECHA OBSOLETO")

    if status_cons != "OBSOLETO" or pd.isna(fecha_entrada) or pd.isna(fecha_OBSOLETO):
        return "FALSO"

    dias_obsolescencia = (fecha_entrada - fecha_OBSOLETO).days

    if dias_obsolescencia <= 90:
        return "1.MENOR DE 90 DIAS"
    elif 90 < dias_obsolescencia <= 180:
        return "2.ENTRE 90 Y 180 DIAS"
    elif 180 < dias_obsolescencia <= 270:
        return "3.ENTRE 180 Y 270 DIAS"
    elif 270 < dias_obsolescencia <= 360:
        return "4.ENTRE 270 Y 360 DIAS"
    elif 360 < dias_obsolescencia <= 540:
        return "5.ENTRE 360 Y 540 DIAS"
    elif 540 < dias_obsolescencia <= 720:
        return "6.ENTRE 540 Y 720 DIAS"
    else:
        return "7.MAYOR DE 720 DIAS"


def calculate_rango_vencido(row: pd.Series) -> str:
    """
    Calcula el rango de vencimiento basado en las fechas de entrada y caducidad.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Rango de vencimiento calculado
    """
    status_cons = row.get("STATUS CONS")
    fecha_entrada = row.get("FECHA ENTRADA")
    fecha_caducidad = row.get("FECH, CADUCIDAD/FECH PREF. CONSUMO")

    if status_cons != "VENCIDO" or pd.isna(fecha_entrada) or pd.isna(fecha_caducidad):
        return "FALSO"

    dias_vencido = (fecha_entrada - fecha_caducidad).days

    if dias_vencido <= 90:
        return "1.MENOR DE 90 DIAS"
    elif 90 < dias_vencido <= 180:
        return "2.ENTRE 90 Y 180 DIAS"
    elif 180 < dias_vencido <= 270:
        return "3.ENTRE 180 Y 270 DIAS"
    elif 270 < dias_vencido <= 360:
        return "4.ENTRE 270 Y 360 DIAS"
    elif 360 < dias_vencido <= 540:
        return "5.ENTRE 360 Y 540 DIAS"
    elif 540 < dias_vencido <= 720:
        return "6.ENTRE 540 Y 720 DIAS"
    else:
        return "7.MAYOR DE 720 DIAS"


def calculate_rango_bloqueado(row: pd.Series) -> str:
    """
    Calcula el rango de bloqueo basado en las fechas de entrada y bloqueo.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Rango de bloqueo calculado
    """
    status_cons = row.get("STATUS CONS")
    fecha_entrada = row.get("FECHA ENTRADA")
    fecha_bloqueado = row.get("FECHA BLOQUEADO")

    if status_cons != "BLOQUEADO" or pd.isna(fecha_entrada) or pd.isna(fecha_bloqueado):
        return "FALSO"

    dias_vencido = (fecha_entrada - fecha_bloqueado).days

    if dias_vencido <= 90:
        return "1.MENOR DE 90 DIAS"
    elif 90 < dias_vencido <= 180:
        return "2.ENTRE 90 Y 180 DIAS"
    elif 180 < dias_vencido <= 270:
        return "3.ENTRE 180 Y 270 DIAS"
    elif 270 < dias_vencido <= 360:
        return "4.ENTRE 270 Y 360 DIAS"
    elif 360 < dias_vencido <= 540:
        return "5.ENTRE 360 Y 540 DIAS"
    elif 540 < dias_vencido <= 720:
        return "6.ENTRE 540 Y 720 DIAS"
    else:
        return "7.MAYOR A 720 DIAS"


def calculate_tiempo_bloqueo(row: pd.Series) -> str:
    """
    Calcula el tiempo de bloqueo basado en las fechas de entrada y bloqueo.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Tiempo de bloqueo calculado
    """
    fecha_entrada = row.get("FECHA ENTRADA")
    fecha_bloqueado = row.get("FECHA BLOQUEADO")
    
    if pd.isna(fecha_entrada) or pd.isna(fecha_bloqueado):
        return 0

    tiempo_bloqueado = (fecha_entrada - fecha_bloqueado).days
    
    return tiempo_bloqueado    


def calculate_rango_cons(row: pd.Series) -> str:
    """
    Calcula el rango de consumo final basado en el status y los rangos correspondientes.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Rango de consumo calculado
    """
    status = row.get("STATUS CONS")

    if status == "OBSOLETO":
        return row.get("RANGO OBSOLESCENCIA")
    elif status == "VENCIDO":
        return row.get("RANGO VENCIDO 2")
    elif status == "BLOQUEADO":
        return row.get("RANGO BLOQUEADO 2")
    elif status == "PAV":
        return row.get("RANGO DE PERMANENCIA 2")
    else:
        return row.get("RANGO DE PERMANENCIA 2")


def lookup_direct(key, df_merge):
    """
    Recorre df_merge y compara la CLAVE 'key' con el valor de la columna 'concatenado'.
    Retorna (factor_prov, clasificacion) si encuentra coincidencia, o (0, 'BAJO') en caso contrario.
    """
    for _, mrow in df_merge.iterrows():
        if str(mrow["concatenado"]).strip() == key:
            return mrow["factor_prov"], mrow["clasificacion"]
    return (0.0, 'BAJO')


def strategy_fpt_condicion1(row, df_merge):
    """
    Si SEGMENTACION es "Marcas propias", "Expertos no locales" o "Dueños de demanda",
    y STATUS CONS es "BLOQUEADO" y TIEMPO BLOQUEO <= 30, retorna (0, 0).
    """
    if (row["SEGMENTACION"] in ["MARCAS PROPIAS", "EXPERTOS NO LOCALES", "DUEÑOS DE CANAL"] and 
        row["STATUS CONS"] == "BLOQUEADO" and row["TIEMPO BLOQUEO"] <= 30):
        return (0.0, 'BAJO')
    return None

def strategy_fpt_condicion2(row, df_merge):
    """
    Si STATUS CONS es "DISPONIBLE" o "PAV", y PERMANENCIA <= 30 y 
    TIPO DE MATERIAL (I) es "GRANEL" o "GRANEL FAB A TERCERO", retorna (0, 'BAJO').
    """
    if (row["STATUS CONS"] in ["DISPONIBLE", "PAV"] and 
        row["PERMANENCIA"] <= 30 and 
        row["TIPO DE MATERIAL (I)"] in ["GRANEL", "GRANEL FAB A TERCERO"]):
        return (0.0, 'BAJO')
    return None

def strategy_fpt_case1(row, df_merge):
    """
    Caso 1: Si NEGOCIO INVENTARIOS es "FPT", se forma la CLAVE concatenando:
        NEGOCIO INVENTARIOS + STATUS CONS + RANGO CONS
    y se busca en df_merge["concatenado"]. Retorna (factor_prov, clasificacion).
    """
    if row["NEGOCIO INVENTARIOS"] == "FPT":
        key = (str(row["NEGOCIO INVENTARIOS"]).strip() +
               str(row["STATUS CONS"]).strip() +
               str(row["RANGO CONS"]).strip())
        return lookup_direct(key, df_merge)
    return None

def strategy_fpt_case2(row, df_merge):
    """
    Caso 2: Si INDICADOR STOCK ESPEC. distinto de "W" y STATUS CONS está en
    ["OBSOLETO", "BLOQUEADO", "VENCIDO"], se forma la CLAVE concatenando:
        NEGOCIO INVENTARIOS + STATUS CONS + RANGO CONS
    y se busca en df_merge["concatenado"]. Retorna (factor_prov, clasificacion).
    """
    if (row["INDICADOR STOCK ESPEC."] != "W" and 
        row["STATUS CONS"] in ["OBSOLETO", "BLOQUEADO", "VENCIDO"]):
        key = (str(row["NEGOCIO INVENTARIOS"]).strip() +
               str(row["STATUS CONS"]).strip() +
               str(row["RANGO CONS"]).strip())
        return lookup_direct(key, df_merge)
    return None


def strategy_1(row, df_merge):
    # Condición 1: Si SEGMENTACIÓN es "DUEÑOS DE CANAL", "MARCAS PROPIAS", "EXPERTOS NO LOCALES"
    # y STATUS CONS es "BLOQUEADO" y TIEMPO BLOQUEO <= 30, entonces se asigna 0%
    if row["SEGMENTACION"] in ["DUEÑOS DE CANAL", "MARCAS PROPIAS", "EXPERTOS NO LOCALES"] \
       and row["STATUS CONS"] == "BLOQUEADO" \
       and row["TIEMPO BLOQUEO"] <= 30:
        return (0.0, 'BAJO')
    return None

def strategy_2(row, df_merge):
    # Condición 2: Si INDICADOR STOCK ESPEC. es "K", entonces 0%
    if row["INDICADOR STOCK ESPEC."] == "K":
        return (0.0, 'BAJO')
    return None

def strategy_3(row, df_merge):
    # Condición 3: Si STATUS CONS es "DISPONIBLE" o "PAV",
    # y PERMANENCIA <= 30 y TIPO DE MATERIAL (I) es "GRANEL" o "GRANEL FAB A TERCERO"
    if row["STATUS CONS"] in ["DISPONIBLE", "PAV"] \
       and row["PERMANENCIA"] <= 30 \
       and row["TIPO DE MATERIAL (I)"] in ["GRANEL", "GRANEL FAB A TERCERO"]:
        return (0.0, 'BAJO')
    return None

def strategy_4(row, df_merge):
    # Condición 4: Si MARCA CONCAT es "AVON" o "NATURA" y STATUS CONS es en {"VENCIDO", "OBSOLETO", "PAV"}
    # se evalúa el primer carácter de RANGO CONS; si > 4, se asigna 100% (muy alto), si <=4 se asigna 20% (mediano)
    if row["MARCA CONCAT"] in ["AVON", "NATURA"] and row["STATUS CONS"] in ["VENCIDO", "OBSOLETO", "PAV"]:
        try:
            first_digit = int(str(row["RANGO CONS"]).strip()[0])
        except:
            first_digit = 0
        if first_digit > 4:
            return (1.0, "MUY ALTO")
        else:
            return (0.2, "MEDIO")
    return None

def strategy_5(row, df_merge):
    # Condición 5: Si MARCA DE QM es "OTRAS" o (STATUS CONS es "DISPONIBLE" y RANGO COBERTURA es "#")
    # se asigna 0%
    if row["MARCA DE QM"] == "OTRAS" or (row["STATUS CONS"] == "DISPONIBLE" and row["RANGO COBERTURA"] == ""):
        return (0.0, 'BAJO')
    return None

def strategy_6(row, df_merge):
    # Condición 6: Si INDICADOR STOCK ESPEC. es "W"
    if row["INDICADOR STOCK ESPEC."] == "W":
        # Dependiendo del STATUS CONS se forma la CLAVE de búsqueda:
        if row["STATUS CONS"] in ["VENCIDO", "PAV"]:
            # CLAVE: concatenar SEGMENTACIÓN, STATUS CONS y RANGO DE PERMANENCIA 2
            key = (str(row["SEGMENTACION"]).strip() + str(row["STATUS CONS"]).strip() + str(row["RANGO DE PERMANENCIA 2"]).strip())
            return lookup_direct(key, df_merge)
        elif row["STATUS CONS"] == "DISPONIBLE":
            # CLAVE: concatenar SEGMENTACIÓN, RANGO COBERTURA y RANGO DE PERMANENCIA 2
            key = (str(row["SEGMENTACION"]).strip() + str(row["RANGO COBERTURA"]).strip() + str(row["RANGO DE PERMANENCIA 2"]).strip())
            return lookup_direct(key, df_merge)
        else:
            # CLAVE: concatenar SEGMENTACIÓN, STATUS CONS y RANGO CONS
            key = (str(row["SEGMENTACION"]).strip() + str(row["STATUS CONS"]).strip() + str(row["RANGO CONS"]).strip())
            return lookup_direct(key, df_merge)
    return None

def strategy_7(row, df_merge):
    # Condición 7: Si INDICADOR STOCK ESPEC. es "Sin asignar" o "O"
    if row["INDICADOR STOCK ESPEC."] in ["SIN ASIGNAR", "O"]:
        if row["STATUS CONS"] == "PAV":
            if row["RANGO PRÓX.VENCER MM"] == "1.PAV 3 MESES":
                # CLAVE: concatenar SEGMENTACIÓN, SUBSEGMENTACION, RANGO COBERTURA y RANGO DE PERMANENCIA 2
                key = (str(row["SEGMENTACION"]).strip() + str(row["SUBSEGMENTACION"]).strip() + str(row["RANGO COBERTURA"]).strip() + str(row["RANGO DE PERMANENCIA 2"]).strip())
                return lookup_direct(key, df_merge)
            elif row["RANGO PRÓX.VENCER MM"] == "2.PAV 4 A 6 MESES":
                key = (str(row["SEGMENTACION"]).strip() + str(row["SUBSEGMENTACION"]).strip() + str(row["RANGO COBERTURA"]).strip() + str(row["RANGO DE PERMANENCIA 2"]).strip())
                return lookup_direct(key, df_merge)
        elif row["STATUS CONS"] == "DISPONIBLE":
            # Intentar primero con una CLAVE y si falla probar otra (SI.ERROR en Excel)
            key = (str(row["SEGMENTACION"]).strip() + str(row["SUBSEGMENTACION"]).strip() + str(row["RANGO COBERTURA"]).strip() + str(row["RANGO DE PERMANENCIA 2"]).strip())
            result = lookup_direct(key, df_merge)
            if result == (0.0, 'BAJO'):
                # CLAVE alternativa: concatenar SEGMENTACION, SUBSEGMENTACION, STATUS CONS, espacio y RANGO CONS
                key_alt = (str(row["SEGMENTACION"]).strip() + str(row["SUBSEGMENTACION"]).strip() + str(row["STATUS CONS"]).strip() + " " + str(row["RANGO CONS"]).strip())
                result = lookup_direct(key_alt, df_merge)
            return result
        elif row["STATUS CONS"] in ["OBSOLETO", "VENCIDO", "BLOQUEADO"]:
            key = (str(row["SEGMENTACION"]).strip() + str(row["SUBSEGMENTACION"]).strip() + str(row["STATUS CONS"]).strip() + " " + str(row["RANGO CONS"]).strip())
            return lookup_direct(key, df_merge)
        else:
            return (0.0, 'BAJO')
    return None

def strategy_default(row, df_merge):
    # Si ninguna de las estrategias previas se aplica, se retorna el valor por defecto (0,0)
    return (0.0, 'BAJO')


def calculate_factor_class(row, df_merge):
    """
    Recibe una fila de df_final_combined y df_merge.
    Recorre las estrategias definidas en orden y retorna la primera que se aplique.
    Si ninguna aplica, retorna (0, 0).
    """
    strategies = [
        strategy_fpt_condicion1,
        strategy_fpt_condicion2,
        strategy_fpt_case1,
        strategy_fpt_case2,
        strategy_1,
        strategy_2,
        strategy_3,
        strategy_4,
        strategy_5,
        strategy_6,
        strategy_7,
        strategy_default
    ]
    for strat in strategies:
        result = strat(row, df_merge)
        if result is not None:
            return result
    return (0.0, 'BAJO')  # Fallback (nunca debería pasar)

def calculate_base_riesgo_column(row: pd.Series) -> float:
    """
    Calcula el valor de la columna de base riesgo
    """
    if row["CLAS BASE RIESGO"] == "BAJO":
        return 0.0
    else:
        return row["VALOR DEF"]

def calculate_provision_column(row: pd.Series) -> float:
    """
    Calcula el valor de la columna de provisión
    """
    if row["MARCA DE QM"] == "OTRAS":
        return 0.0
    else:
        return row["VALOR DEF"] * row['FACTOR PROV']


def process_dataframe(df_final_combined: pd.DataFrame) -> pd.DataFrame:
    """
    Procesa el DataFrame aplicando todas las reglas de negocio en el orden específico requerido.

    Args:
        df: DataFrame con los datos de SAP

    Returns:
        pd.DataFrame: DataFrame procesado con todas las columnas calculadas
    """
    # 1. Añadir columnas formuladas de 'MARCA CONCAT' y 'SEGMENTACION'
    df_final_combined["MARCA CONCAT"] = df_final_combined["MARCA DE QM"].apply(lambda x: insert_marks().get(x, ""))
    df_final_combined["SEGMENTACION"] = df_final_combined["MARCA DE QM"].apply(
        lambda x: insert_segments().get(x, "OTRAS")
    )
    df_final_combined["SUBSEGMENTACION"] = df_final_combined["MARCA DE QM"].apply(
        lambda x: insert_subsegmentacion().get(x, "")
    )

    # 2. Calcular 'RANGO DE PERMANENCIA 2'
    required_columns = {"LOTE", "PERMANENCIA", "RANGO DE PERMANENCIA"}
    if required_columns.issubset(df_final_combined.columns):
        df_final_combined["RANGO DE PERMANENCIA 2"] = df_final_combined.apply(calculate_rango_permanencia, axis=1)
    else:
        print(f"Faltan columnas: {required_columns - set(df_final_combined.columns)}")

    # 3. Calcular 'STATUS CONS'
    required_columns = {"RANGO PRÓX.VENCER MM", "VALOR BLOQUEADO MM", "VALOR OBSOLETO"}
    if required_columns.issubset(df_final_combined.columns):
        df_final_combined["STATUS CONS"] = df_final_combined.apply(calculate_status_cons, axis=1)
    else:
        print(f"Faltan columnas: {required_columns - set(df_final_combined.columns)}")

    # 4. Calcular 'VALOR DEF'
    required_columns = {"STATUS CONS", "VALOR BLOQUEADO MM", "VALOR TOTAL MM"}
    if required_columns.issubset(df_final_combined.columns):
        df_final_combined["VALOR DEF"] = df_final_combined.apply(calculate_valor_def, axis=1)
    else:
        print(f"Faltan columnas: {required_columns - set(df_final_combined.columns)}")

    # 5. Reemplazar valores inválidos
    df_final_combined.replace("#", np.nan, inplace=True)

    # 6. Convertir columnas de fecha
    date_columns = [
        "FECHA ENTRADA",
        "FECHA OBSOLETO",
        "FECHA BLOQUEADO",
        "FECH. FABRICACIÓN",
        "CREADO EL",
        "FECH, CADUCIDAD/FECH PREF. CONSUMO",
    ]

    for col in date_columns:
        if col in df_final_combined.columns:
            df_final_combined[col] = pd.to_datetime(df_final_combined[col], format="%d/%m/%Y", errors="coerce")

    # 7. Calcular 'RANGO OBSOLESCENCIA'
    df_final_combined["RANGO OBSOLESCENCIA"] = df_final_combined.apply(calculate_rango_obsolescencia, axis=1)

    # 8. Calcular 'RANGO VENCIDO 2'
    df_final_combined["RANGO VENCIDO 2"] = df_final_combined.apply(calculate_rango_vencido, axis=1)

    # 9. Calcular 'RANGO BLOQUEADO 2'
    df_final_combined["RANGO BLOQUEADO 2"] = df_final_combined.apply(calculate_rango_bloqueado, axis=1)

    # 10. Calcular 'RANGO CONS'
    df_final_combined["RANGO CONS"] = df_final_combined.apply(calculate_rango_cons, axis=1)
    
    # 11. Calcular 'TIEMPO BLOQUEO'
    df_final_combined["TIEMPO BLOQUEO"] = df_final_combined.apply(calculate_tiempo_bloqueo, axis=1)

    # 12. Calcular 'FACTOR' y 'CLASIFICACIÓN'
    results = df_final_combined.apply(lambda row: calculate_factor_class(row, df_merge), axis=1)
    df_final_combined["FACTOR PROV"] = results.apply(lambda tup: tup[0])
    df_final_combined["CLAS BASE RIESGO"] = results.apply(lambda tup: tup[1])

    # 13. Calcular 'BASE RIESGO'
    df_final_combined["BASE RIESGO"] = df_final_combined.apply(calculate_base_riesgo_column, axis=1)
    
    # 14. Calcular 'PROVISION'
    df_final_combined["PROVISION"] = df_final_combined.apply(calculate_provision_column, axis=1)
    
    # Columnas en el orden deseado
    columnas_ordenadas = [
        "NEGOCIO INVENTARIOS", "AÑO NATURAL/MES","TIPO MATERIAL INVENTARIO", "MARCA DE QM", "MATERIAL", 
        "DESCRIPCIÓN", "UNIDAD MEDIDA", "CENTRO", "CODIGO ALMACEN CLIENTE", "INDICADOR STOCK ESPEC.",
        "NÚM.STOCK.ESP.", "LOTE", "CREADO EL", "FECH. FABRICACIÓN", "FECH, CADUCIDAD/FECH PREF. CONSUMO",
        "FECHA BLOQUEADO", "FECHA OBSOLETO", "FECHA ENTRADA", "RANGO OBSOLETO 2", "RANGO COBERTURA",
        "RANGO DE PERMANENCIA", "RANGO BLOQUEADO", "RANGO OBSOLETO", "RANGO VENCIDOS", "PRÓXIMO A VENCER",
        "RANGO PRÓX.VENCER MM", "RANGO PRÓXIMOS A VEN", "TIPO DE MATERIAL (I)", "COSTO UNITARIO REAL",
        "INVENTARIO DISPONIBL", "INVENTARIO NO DISPON", "VALOR OBSOLETO", "VALOR BLOQUEADO MM", "VALOR TOTAL MM", "PERMANENCIA",

        "MARCA CONCAT", "SEGMENTACION", "SUBSEGMENTACION",  
        "RANGO DE PERMANENCIA 2",
        "STATUS CONS", "VALOR DEF", "RANGO OBSOLESCENCIA", "RANGO VENCIDO 2",
        "RANGO BLOQUEADO 2", "RANGO CONS", "TIEMPO BLOQUEO", 
        "FACTOR PROV", "CLAS BASE RIESGO", "BASE RIESGO", "PROVISION" 
    ]
    
    return df_final_combined